# <u>Order Details Analysis</u>

## About
Boss Leathers is a small leather products business which has recently started selling its products on Amazon. Currently, it has around 40 SKUs registered in the Indian Marketplace. Over the past few months, it has incurred some loss due to return orders. Now, Boss Leather seeks help to predict the liklihood of a new order being rejected. This would help them to take necessary actions and subsequently reduce the loss.

## Objective
To build a model which would predict the order status (`Delivered to buyer` or `Returned to seller`)

## Data Dictionary
The Order data is provided in an excel file. The columns are:
Independent Features:
   1. `order_no` - Unique Amazon Order Number
   
   2. `order_date` - Date on which the order was placed
   
   4. `buyer` - Name of the buyer
   
   3. `ship_city` - Delivery Address City
   
   4. `ship_state` - Delivery Address State
   
   5. `sku - Unique` identifier of a product
   
   6. `description` - Product description
   
   7. `quantity` - Number of units ordered
   
   8. `item_total` - Total amount paid by the buyer
   
   9. `shipping_fee` - Charges borne by Boss Leathers to ship the item
   
   10. `cod` - Mode of payment: Cash on delivery or not
   
Label / Target Feature:
   1. `order_status` - Status of the order

# Analysis Steps:
   - Import libraries
   - Understand the data
   - Handle missing values
   - Format Date Feature
   - Visualize Business Insigths

### Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import plotly.express as px
warnings.simplefilter('ignore')

from wordcloud import WordCloud, STOPWORDS
import nltk
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.corpus import stopwords

%matplotlib inline

In [ ]:
!pip install openpyxl

In [ ]:
df = pd.read_excel('../input/amazon-seller-order-status-prediction/orders_data.xlsx')

In [ ]:
df.shape

In [ ]:
def drop(df, *features):
    ''' To drop the features from a dataframe'''
    for i in features:
        df.drop(i, axis=1, inplace=True)

In [ ]:
df.info()

In [ ]:
df.head(3)

In [ ]:
# Convert city and state into uppercase
places = ['ship_city', 'ship_state']
for i in places:
    df[i] = df[i].apply(lambda x: x.upper())

In [ ]:
# removing comma from city values
df['ship_city'] = df['ship_city'].apply(lambda x: x.replace(',', ''))
df['ship_state'] = df['ship_state'].apply(lambda x: x.replace(',', ''))

In [ ]:
df.isnull().sum()

### Handle Missing Values
There are 3 features with missing values: cod, item_total and shipping fee

In [ ]:
# imputing with mode as shipping fee is fixed based on package size and weight
df['shipping_fee'].fillna(df['shipping_fee'].mode()[0], inplace=True)

# imputing with mode as we are considering the item that is sold the most
df['item_total'].fillna(df['item_total'].mode()[0], inplace=True)

# assuming if an order is not cod would mean the mode of payment is online
df['cod'].fillna('online', inplace=True)

In [ ]:
# remove rupee symbol from amount features
amounts = ['item_total', 'shipping_fee']
for i in amounts:
    df[i] = df[i].apply(lambda x: x.replace(',', ''))        # remove the commas in amounts greater than 1000
    df[i] = df[i].apply(lambda x: x[1:])

In [ ]:
# change data types
i = 'int64'
f = 'float64'
df = df.astype({'item_total': f, 'shipping_fee': f, 'quantity': i})

In [ ]:
df.head(3)

## <u>Drawing Business Insights:</u>

We will try to answer few questions:
   - Is order success dependant on mode of payment?
   - Sales Trend (Sales Pattern over the period)
   - Which days of the week draw more sales?
   - People from which states are ordering the most
   - Which are the top sold products?
   - What percentage of orders have been returned?
   - Most popular keywords among the customers

### cod distribution

In [ ]:
df[['cod', 'order_status', 'item_total']].groupby(['cod', 'order_status']).sum().reset_index()

In [ ]:
fig = px.histogram(df, x="cod", y="item_total", color='order_status', barmode='group', height=400)
fig.show()

We see that most of the orders were prepaid and the percentage of return orders is more in cod mode.

### Format date

In [ ]:
# extract year, month, day and time from date feature
df['year'] = pd.DatetimeIndex(df['order_date']).year
df['date'] = pd.DatetimeIndex(df['order_date']).date
df['time'] = pd.DatetimeIndex(df['order_date']).time
df['month_name'] = pd.DatetimeIndex(df['date']).month_name()
df['day_name'] = pd.DatetimeIndex(df['date']).day_name()

In [ ]:
drop(df, 'order_date')
df.head(2)

In [ ]:
# Extracting the unique identifier
df['sku'] = df['sku'].apply(lambda x: x[4:])
df.head(2)

In [ ]:
df_sales = df[['date', 'item_total']].groupby('date').sum().reset_index()
df_sales.head(3)

In [ ]:
fig = px.line(df_sales, x='date', y="item_total", title = 'Sales over the period')
fig.show()

There hasn't been any significant growth over the period. December month saw the maximum sales with a couple of breakthroughs. 

In [ ]:
df_days = df[['year', 'month_name', 'day_name', 'item_total']].groupby(['year', 'month_name', 'day_name']).sum().reset_index()
# df_days.sort_values(by=['year', 'month_name'])

In [ ]:
fig = px.bar(df_days, x="month_name", y="item_total", color="day_name", title="Month-wise Sales")
fig.show()

In [ ]:
df_places = df[['ship_state', 'ship_city', 'item_total']].groupby(['ship_state', 'ship_city']).sum().reset_index()
df_places.sort_values(by='item_total', ascending=False, inplace=True)

In [ ]:
fig = px.sunburst(data_frame=df_places, path=['ship_state', 'ship_city'], values='item_total', title='Region-wise orders')
fig.show()

In [ ]:
top_cities = df.groupby('ship_city').size().reset_index().rename(columns={0: 'Total'}).sort_values('Total', ascending=False).head()
fig = px.pie(top_cities, values='Total', names='ship_city', color_discrete_sequence=px.colors.sequential.RdBu, title='Top 5 ordering cities')
fig.show()

In [ ]:
top_cities = df.groupby('sku').size().reset_index().rename(columns={0: 'Total'}).sort_values('Total', ascending=False).head()
fig = px.pie(top_cities, values='Total', names='sku', color_discrete_sequence=px.colors.sequential.BuGn_r, title='Top 5 Products')
fig.show()

In [ ]:
df['order_status'].value_counts()

In [ ]:
df_status = df[['order_status', 'item_total']]
fig = px.bar(data_frame=df_status, x='order_status', y='item_total', color='order_status', title='Success rate of orders')
fig.show()

From the above we can find the order value of each of success and return orders

In [ ]:
df['description'] = df['description'].apply(lambda x: x.replace('|', ''))

In [ ]:
stop_words = stopwords.words('english')
stop_words.extend(['Pure', 'Leather', 'Shantiniketan', 'with', 'clean_dedsc', 'phone', 'for', 'and', 'print', 'block', 'dtype'])

In [ ]:
# Remove stop words and remove words with 2 or less characters
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 2 and token not in stop_words:
            result.append(token)
            
    return ' '.join(result)

In [ ]:
# cleaned data
df['clean_desc'] = df['description'].apply(preprocess)

In [ ]:
plt.figure(figsize=(15,10))
wc = WordCloud(max_words=2000, width=1600, height=800, stopwords=stop_words).generate(str(df[df['order_status']=='Delivered to buyer'].clean_desc))
plt.title('Most Popular words', fontsize=20)
plt.imshow(wc)

## Conclusions

We have extracted quite a few key inferences:
   - The dataset came with some missing values in 2 of its features
   - Order return rates are higher in COD type orders
   - The sales has been stagnant throughout with few breakthroughs in December 
   - We visualized the sales across the months grouped by day of week to find that Sundays and Wednesdays dominate over the sales whereas the least contributor turned out to be the Saturdays
   - We saw the region wise order amounts
   - Top ordering cities are: Mumbai, Kolkata, Bangalore, Chennai and Pune
   - We identified the top selling products
   - The Barplot of order status tells us that there is less than 10% of return orders
   - In the word cloud we see which are the most popular categories among the customers

**Further Analysis in progress**